In [14]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import time
from mtcnn import MTCNN
from keras.models import model_from_json
from keras.preprocessing import image

In [5]:
#mtcnn
mtcnn_detector = MTCNN()

#--------------------------------------------

cap = cv2.VideoCapture(0)

#--------------------------------------------

tic = time.time()
frame = 0
while(True):

    if frame % 100 == 0:
        toc = time.time()
        print(frame,", ",toc-tic)
        tic = time.time()

    ret, img = cap.read()
    
    try:
        original_size = img.shape

        cv2.putText(img, 'mtcnn', (70, 120), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2)

        #img = dlib.load_rgb_image("img1.jpg")

        detections = mtcnn_detector.detect_faces(img)

        for detection in detections:
            confidence_score = str(round(100*detection["confidence"], 2))+"%"
            x, y, w, h = detection["box"]

            cv2.putText(img, confidence_score, (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

            cv2.rectangle(img, (x, y), (x+w, y+h),(255,255,255), 1) #highlight detected face

        #----------------------------

        cv2.imshow('img',img)
        frame = frame + 1
    except Exception as e:
        quit = True
        print(str(e))

    if cv2.waitKey(1) & 0xFF == ord('q'):  # press q to quit
        break


#kill open cv things
cap.release()
cv2.destroyAllWindows()

0 ,  0.0
100 ,  45.189950942993164


In [16]:
Song_list = pd.read_excel('./Song_list.xlsx')

# -----------------------------
# opencv initialization

mtcnn_detector = MTCNN()

# -----------------------------
# face expression recognizer initialization

model = model_from_json(open("./model/facial_expression_model_structure.json", "r").read())
model.load_weights('./model/facial_expression_model_weights.h5')  # load weights
# -----------------------------

emotions = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')

cap = cv2.VideoCapture(0)  # process real time web-cam

frame = 0

emotion_list = []

while (True):
    ret, img = cap.read()

    original_size = img.shape

    cv2.putText(img, 'mtcnn', (70, 120), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2)

    #img = dlib.load_rgb_image("img1.jpg")

    detections = mtcnn_detector.detect_faces(img)

    for detection in detections:
        confidence_score = str(round(100*detection["confidence"], 2))+"%"
        x, y, w, h = detection["box"]

        cv2.putText(img, confidence_score, (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

        cv2.rectangle(img, (x, y), (x+w, y+h),(255,255,255), 1) #highlight detected face

        detected_face = img[int(y):int(y + h), int(x):int(x + w)]  # crop detected face
        detected_face = cv2.cvtColor(detected_face, cv2.COLOR_BGR2GRAY)  # transform to gray scale
        detected_face = cv2.resize(detected_face, (48, 48))  # resize to 48x48

        img_pixels = image.img_to_array(detected_face)
        img_pixels = np.expand_dims(img_pixels, axis=0)

        img_pixels /= 255  # pixels are in scale of [0, 255]. normalize all pixels in scale of [0, 1]

        # ------------------------------

        predictions = model.predict(img_pixels)  # store probabilities of 7 expressions
        max_index = np.argmax(predictions[0])

        # background of expression list
        overlay = img.copy()
        opacity = 0.4
        cv2.rectangle(img, (x + w + 10, y - 25), (x + w + 150, y + 115), (64, 64, 64), cv2.FILLED)
        cv2.addWeighted(overlay, opacity, img, 1 - opacity, 0, img)

        # connect face and expressions
        cv2.line(img, (int((x + x + w) / 2), y + 15), (x + w, y - 20), (255, 255, 255), 1)
        cv2.line(img, (x + w, y - 20), (x + w + 10, y - 20), (255, 255, 255), 1)

        emotion = ""

        for i in range(len(predictions[0])):
            emotion = "%s %s%s" % (emotions[i], round(predictions[0][i] * 100, 2), '%')
            j = np.argmax(predictions[0])
            k = emotions[j]
            emotion_list.append(k)
            """if i != max_index:
                color = (255,0,0)"""

            color = (255, 255, 255)

            cv2.putText(img, emotion, (int(x + w + 15), int(y - 12 + i * 20)), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                        color, 1)

            # -------------------------

    cv2.imshow('img', img)

    frame = frame + 1
    # print(frame)

    # ---------------------------------

    if frame > 50:
        break

    if cv2.waitKey(1) & 0xFF == ord('q'):  # press q to quit
        break

# kill open cv things
cap.release()
cv2.destroyAllWindows()